# A Meeting Analysis Use Case

Copyright 2025, Denis Rothman

The notebook's goal is to demonstrate how to guide an AI through a multi-step analytical process, moving from a raw transcript to actionable insights, thereby training both the user and the AI.

This notebook is a step-by-step guide on how you can engineer a structured context . This guide will serve as the architectural blueprint for your code, explaining not just the *what* but the *why* at each stage.


-----

### Setup and Preliminaries

This section handles the basic configuration.

1.  **Cell 1: Install Libraries**

      * Install the necessary OpenAI library.

In [1]:
# Cell 1: Installation
!pip install tqdm==4.67.1 --upgrade
!pip install openai==2.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


2.  **Cell 2: Imports and API Key**

      * Import the library and securely prompt for the user's API key. This is better than hard-coding it.

In [2]:
# Cell 2: Imports and API Key Setup
# We will use the OpenAI library to interact with the LLM and Google Colab's
# secret manager to securely access your API key.

import os
from openai import OpenAI
from google.colab import userdata

# Load the API key from Colab secrets, set the env var, then init the client
try:
    api_key = userdata.get("API_KEY")
    if not api_key:
        raise userdata.SecretNotFoundError("API_KEY not found.")

    # Set environment variable for downstream tools/libraries
    os.environ["OPENAI_API_KEY"] = api_key

    # Create client (will read from OPENAI_API_KEY)
    client = OpenAI()
    print("OpenAI API key loaded and environment variable set successfully.")

except userdata.SecretNotFoundError:
    print('Secret "API_KEY" not found.')
    print('Please add your OpenAI API key to the Colab Secrets Manager.')
except Exception as e:
    print(f"An error occurred while loading the API key: {e}")


OpenAI API key loaded and environment variable set successfully.


3.  **Cell 3: The Raw Data (The "Crime Scene")**

      * Define the meeting transcript as a multi-line string. This is our primary data source.

In [3]:
# Cell 3: The Full Meeting Transcript
meeting_transcript = """
        Tom: Morning all. Coffee is still kicking in.
        Sarah: Morning, Tom. Right, let's jump in. Project Phoenix timeline. Tom, you said the backend components are on track?
        Tom: Mostly. We hit a small snag with the payment gateway integration. It's... more complex than the docs suggested. We might need another three days.
        Maria: Three days? Tom, that's going to push the final testing phase right up against the launch deadline. We don't have that buffer.
        Sarah: I agree with Maria. What's the alternative, Tom?
        Tom: I suppose I could work over the weekend to catch up. I'd rather not, but I can see the bind we're in.
        Sarah: Appreciate that, Tom. Let's tentatively agree on that. Maria, what about the front-end?
        Maria: We're good. In fact, we're a bit ahead. We have some extra bandwidth.
        Sarah: Excellent. Okay, one last thing. The marketing team wants to do a big social media push on launch day. Thoughts?
        Tom: Seems standard.
        Maria: I think that's a mistake. A big push on day one will swamp our servers if there are any initial bugs. We should do a soft launch, invite-only for the first week, and then do the big push. More controlled.
        Sarah: That's a very good point, Maria. A much safer strategy. Let's go with that. Okay, great meeting. I'll send out a summary.
        Tom: Sounds good. Now, more coffee.
        """

-----

### Layer 1: Establishing the scope (The 'What')

Here, we define the scope of the analysis. Each step's output will inform the next, creating a chain of context.

1.  **Cell 4: g2 - Set the Signal-to-Noise Ratio**

      * We'll start by cleaning the data. The prompt explicitly tells the AI to separate substantive content from conversational noise.

In [4]:
# Cell 4: g2 - Isolating Content from Noise
prompt_g2 = f"""
        Analyze the following meeting transcript. Your task is to isolate the substantive content from the conversational noise.
        - Substantive content includes: decisions made, project updates, problems raised, and strategic suggestions.
        - Noise includes: greetings, pleasantries, and off-topic remarks (like coffee).
        Return ONLY the substantive content.

        Transcript:
        ---
        {meeting_transcript}
        ---
        """

In [5]:
from openai import OpenAI

try:
    client = OpenAI()

    response_g2 = client.chat.completions.create(
        model="gpt-5.1",
        messages=[
            {"role": "user", "content": prompt_g2}
        ]
    )

    substantive_content = response_g2.choices[0].message.content
    print("--- SUBSTANTIVE CONTENT ---")
    print(substantive_content)

except Exception as e:
    print(f"An error occurred: {e}")

--- SUBSTANTIVE CONTENT ---
Sarah: Project Phoenix timeline. Tom, you said the backend components are on track?  
Tom: Mostly. We hit a small snag with the payment gateway integration. It's more complex than the docs suggested. We might need another three days.  
Maria: Three days? Tom, that's going to push the final testing phase right up against the launch deadline. We don't have that buffer.  
Sarah: I agree with Maria. What's the alternative, Tom?  
Tom: I suppose I could work over the weekend to catch up. I'd rather not, but I can see the bind we're in.  
Sarah: Appreciate that, Tom. Let's tentatively agree on that. Maria, what about the front-end?  
Maria: We're good. In fact, we're a bit ahead. We have some extra bandwidth.  
Sarah: Excellent. Okay, one last thing. The marketing team wants to do a big social media push on launch day. Thoughts?  
Tom: Seems standard.  
Maria: I think that's a mistake. A big push on day one will swamp our servers if there are any initial bugs. We 

2.  **Cell 5: g3 - Define the Scope of Time (Simulated RAG)**

      * We'll simulate a RAG context by providing a "previous" summary and asking for what's new. This teaches the user the importance of historical context.

In [6]:
# Cell 5: g3 - Identifying NEW Information (Simulated RAG)
previous_summary = "In our last meeting, we finalized the goals for Project Phoenix and assigned backend work to Tom and front-end to Maria."

prompt_g3 = f"""
Context: The summary of our last meeting was: "{previous_summary}"

Task: Analyze the following substantive content from our new meeting. Identify and summarize ONLY the new developments, problems, or decisions that have occurred since the last meeting.

New Meeting Content:
---
{substantive_content}
---
"""
from openai import OpenAI
from google.colab import userdata


# Your chat completion request
try:
    response_g3 = client.chat.completions.create(
        model="gpt-5.1",
        messages=[{"role": "user", "content": prompt_g3}]
    )
    new_developments = response_g3.choices[0].message.content
    print("--- NEW DEVELOPMENTS SINCE LAST MEETING ---")
    print(new_developments)
except Exception as e:
    print(f"An error occurred: {e}")


--- NEW DEVELOPMENTS SINCE LAST MEETING ---
New developments, problems, and decisions since the last meeting:

1. **Backend delay and workaround**
   - New problem: The payment gateway integration is more complex than expected, causing a potential delay of about three days.
   - Decision: To avoid slipping the overall timeline, Tom will likely work over the weekend to keep the backend on track (tentatively agreed).

2. **Front-end status**
   - New development: Front-end work is not just on track but slightly ahead, and Maria reports extra bandwidth.

3. **Launch and marketing strategy change**
   - New proposal/problem: Concern raised that a large social media push on launch day could overwhelm servers and expose initial bugs.
   - Decision: Shift from a big public launch to a **soft launch** (invite-only for the first week), followed by the major social media push afterward.


-----

### Layer 2: Conducting the Investigation (The 'How')

Now we move from identifying facts to generating insights, the core of the semantic context interpreation journey.

1.  **Cell 6: g4 - Identify the Key Threads**

      * This is a crucial step. The prompt asks the AI to read between the lines.

In [7]:
# Cell 6: g4 - Uncovering Implicit Threads
prompt_g4 = f"""
Task: Analyze the following meeting content for implicit social dynamics and unstated feelings. Go beyond the literal words.
- Did anyone seem hesitant or reluctant despite agreeing to something?
- Were there any underlying disagreements or tensions?
- What was the overall mood?

Meeting Content:
---
{substantive_content}
---
"""
try:
    response_g4 = client.chat.completions.create(
        model="gpt-5.1",
        messages=[{"role": "user", "content": prompt_g4}]
    )
    implicit_threads = response_g4.choices[0].message.content
    print("--- IMPLICIT THREADS AND DYNAMICS ---")
    print(implicit_threads)
except Exception as e:
    print(f"An error occurred: {e}")

--- IMPLICIT THREADS AND DYNAMICS ---
Here’s what seems to be going on beneath the surface:

1. **Hesitation / Reluctance**
   - **Tom and weekend work:**  
     - He says, “I suppose I could work over the weekend… I’d rather not, but I can see the bind we’re in.”  
     - The phrasing (“I suppose,” “I’d rather not”) shows clear reluctance. He’s agreeing, but it sounds like a pressured concession rather than a voluntary commitment.
     - Sarah’s response, “Appreciate that, Tom. Let’s tentatively agree on that,” treats his offer as the solution without exploring alternatives, suggesting he may feel boxed in.

2. **Underlying Disagreements / Tensions**
   - **Schedule pressure:**  
     - Maria’s reaction (“Three days?... we don’t have that buffer.”) is firm and a bit sharp. There’s a subtle tension between:
       - Tom’s need for realistic technical time (integration complexity).
       - Maria’s focus on the overall project timeline and launch risk.
     - This is not an open conflic

2.  **Cell 7: g5 - Perform "Intellectual Combinations"**

      * Here, we prompt the AI to be creative and solve a problem by synthesizing different ideas from the meeting.

In [8]:
# Cell 7: g5 - Generating a Novel Solution
prompt_g5 = f"""
Context: In the meeting, Maria suggested a 'soft launch' to avoid server strain, and also mentioned her team has 'extra bandwidth'.
Tom is facing a 3-day delay on the backend.

Task: Propose a novel, actionable idea that uses Maria's team's extra bandwidth to help mitigate Tom's 3-day delay. Combine these two separate pieces of information into a single solution.
"""
try:
    response_g5 = client.chat.completions.create(
        model="gpt-5.1",
        messages=[{"role": "user", "content": prompt_g5}]
    )
    novel_solution = response_g5.choices[0].message.content
    print("--- NOVEL SOLUTION PROPOSED BY AI ---")
    print(novel_solution)
except Exception as e:
    print(f"An error occurred: {e}")

--- NOVEL SOLUTION PROPOSED BY AI ---
Have Maria’s team use their “extra bandwidth” to run and support an extended soft launch that partially bypasses Tom’s 3‑day backend delay.

Concrete plan:

1. **Define a limited feature set that doesn’t depend on Tom’s delayed pieces**  
   - Maria’s team identifies which user flows can run on mock data, feature flags, or temporary workarounds (e.g., cached responses, stubbed APIs, or reduced functionality).  
   - Tom flags the endpoints or features that are delayed; everything else is fair game for the soft launch.

2. **Maria’s team builds a “soft-launch compatibility layer”**  
   Using their extra bandwidth, Maria’s team:  
   - Implements temporary mock services / stubs for the delayed backend endpoints.  
   - Adds feature flags so the app can switch from mock to real backend with a config change once Tom is done.  
   - Adjusts the UI/UX to gracefully degrade where data is incomplete (e.g., “Detailed stats available in 3 days”).

3. **Run 

-----

### Layer 3: Determining the Action (The 'What Next')

Finally, we turn the analysis into concrete, forward-looking artifacts.

1.  **Cell 8: g6 - Define the Output Format (Final Summary)**

      * We compile all the key information into a structured, final summary, showing the importance of clear outputs.

In [9]:
# Cell 8: g6 - Creating the Final, Structured Summary
prompt_g6 = f"""
Task: Create a final, concise summary of the meeting in a markdown table.
Use the following information to construct the table.

- New Developments: {new_developments}

The table should have three columns: "Topic", "Decision/Outcome", and "Owner".
"""
try:
    response_g6 = client.chat.completions.create(
        model="gpt-5.1",
        messages=[{"role": "user", "content": prompt_g6}]
    )
    final_summary_table = response_g6.choices[0].message.content
    print("--- FINAL MEETING SUMMARY TABLE ---")
    print(final_summary_table)
except Exception as e:
    print(f"An error occurred: {e}")

--- FINAL MEETING SUMMARY TABLE ---
| Topic                              | Decision/Outcome                                                                                          | Owner  |
|------------------------------------|-----------------------------------------------------------------------------------------------------------|--------|
| Backend delay & payment gateway    | Payment integration more complex than expected; Tom to likely work over the weekend to avoid timeline slip | Tom    |
| Front-end status                   | Front-end is slightly ahead of schedule; Maria has extra bandwidth                                       | Maria  |
| Launch & marketing strategy change | Move from big public launch to a one-week invite-only soft launch, then major social media push afterward | Team / Marketing Lead


2.  **Cell 9: g7 - Generate the Subsequent Task**

      * The last step is to use the analysis to perform a real-world action, closing the loop from insight to action.

In [10]:
# Cell 9: g7 - Drafting the Follow-Up Action
prompt_g7 = f"""
Task: Based on the following summary table, draft a polite and professional follow-up email to the team (Sarah, Tom, Maria).
The email should clearly state the decisions made and the action items for each person.

Summary Table:
---
{final_summary_table}
---
"""
try:
    response_g7 = client.chat.completions.create(
        model="gpt-5.1",
        messages=[{"role": "user", "content": prompt_g7}]
    )
    follow_up_email = response_g7.choices[0].message.content
    print("--- DRAFT FOLLOW-UP EMAIL ---")
    print(follow_up_email)
except Exception as e:
    print(f"An error occurred: {e}")

--- DRAFT FOLLOW-UP EMAIL ---
Subject: Recap: Decisions & Next Steps from Today’s Meeting

Hi Sarah, Tom, and Maria,

Following up on our discussion, here is a quick summary of decisions and action items:

1. Backend Delay & Payment Gateway  
   - Decision: The payment integration is more complex than expected, and we want to avoid slipping the overall timeline.  
   - Action (Tom): You’ll review the remaining integration work and, if necessary, work over the weekend to keep us on track. Please let us know by tomorrow if you anticipate needing additional support or any scope adjustments.

2. Front-End Status  
   - Decision: The front-end is slightly ahead of schedule, giving us some extra capacity.  
   - Action (Maria): You’ll continue progressing the front-end and be available to support backend-related UI changes or integration tasks as needed. Please coordinate directly with Tom on priorities.

3. Launch & Marketing Strategy  
   - Decision: We will shift from a big public launch 

This notebook structure doesn't just "get a summary." It takes the user on a journey, showing them *how* to think with the AI as a partner. It perfectly translates the abstract "Scope, Investigation, Action" framework into a repeatable, educational, and powerful process.